In [3]:
%pip install pandas

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------  11.5/11.6 MB 60.3 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 55.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   ------------------------------- -------- 12.6/15.9 MB 56.5 MB/s eta 0:00:01
   ---------------------------------------- 15.9/15.9 MB 52.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import json
with open('test_instructions.json', 'r') as file:
    data = json.load(file)

all_data = []
for category, items in data.items():
    for item in items:
        item['category'] = category  # 카테고리 정보 추가
        all_data.append(item)

In [35]:
import re
import json

def extract_json_list_from_response(content):
    if isinstance(content, list):
        return content
    if not isinstance(content, str):
        raise TypeError("Expected string or list as input")

    # 1. Try to extract JSON inside triple backticks
    code_block_match = re.search(r"```json\s*(.*?)```", content, re.DOTALL)
    if code_block_match:
        json_str = code_block_match.group(1)
    else:
        bracket_match = re.search(r"\[.*\]", content, re.DOTALL)
        if bracket_match:
            json_str = bracket_match.group(0)
        else:
            raise ValueError("No JSON-like list found in response.")

    # 2. Escape inner double quotes inside each string entry
    # Step 1: Split into individual entries
    entries = re.findall(r'"(.*?)"', json_str, re.DOTALL)
    if not entries:
        raise ValueError("Failed to extract entries from raw string.")

    # Step 2: Escape internal quotes and rebuild the list
    cleaned_entries = []
    for entry in entries:
        # Replace internal " with \", and ensure it's a valid string
        cleaned = entry.replace('\\"', '"')  # Remove any pre-escaped quotes
        cleaned = cleaned.replace('"', '\\"')  # Escape any remaining quotes
        cleaned_entries.append(f'"{cleaned}"')

    cleaned_json = "[" + ",\n".join(cleaned_entries) + "]"

    # 3. Parse JSON
    return json.loads(cleaned_json)


In [36]:
from openai import OpenAI
import time
import json
from tqdm import tqdm
import os
from dotenv import load_dotenv
load_dotenv()
def generate_similar_prompts(base_prompt, n=10):
    system_prompt = (
        "You are an assistant that creates diverse, non-redundant prompts for PowerPoint slide editing. "
        "Given a base prompt, generate {} alternative prompts that are functionally similar, but use different expressions or target different languages. "
        "Ensure the results are distinct and not trivial rephrases.".format(n)
    )

    user_prompt = f"Base prompt: {base_prompt}\n\nReturn a JSON list of {n} different, unique prompts. Give me a JSON."

    client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)   
    retries = 3
    for attempt in range(retries):
        try:
            response = client.responses.create(
                model="gpt-4o",
                instructions = system_prompt,
                input = user_prompt,
                temperature=0.2
            )
            content = response.output_text
            #print(content)
            try:
                generated_prompts = extract_json_list_from_response(content)
                if isinstance(generated_prompts, list) and len(generated_prompts) == n:
                    return generated_prompts
                else:
                    raise ValueError("Invalid response structure")
            except json.JSONDecodeError:
                raise ValueError("Could not parse JSON")
        except Exception as e:
            print(f"Attempt {attempt+1} failed: {e}")
            time.sleep(2)
    return []


In [37]:

# Example: all_data = [{"prompt": "Translate all visible text elements on ppt slide number {slide_num} into Japanese."}, ...]

output = []

for i in tqdm(range(len(all_data))):
    input_prompt = all_data[i]['prompt']
    generated = generate_similar_prompts(input_prompt, n=10)
    output.append({
        "original": input_prompt,
        "generated_variants": generated
    })

# Save to file
with open("augmented_prompts.json", "w", encoding="utf-8") as f:
    json.dump(output, f, indent=2, ensure_ascii=False)


 14%|█▍        | 8/58 [00:29<03:20,  4.01s/it]

Attempt 1 failed: Invalid response structure
Attempt 2 failed: Invalid response structure


100%|██████████| 58/58 [04:20<00:00,  4.50s/it]


In [27]:
len(all_data)

58

58 * 10